##Downloading Spark, settings env variables

In [ ]:
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark


import os
os.environ['OBJC_DISABLE_INITIALIZE_FORK_SAFETY'] = 'YES'
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.

##Initializing Spark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
print("Sucessful")

Sucessful


In [ ]:
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, udf
from functools import reduce
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
import numpy as np

import re
import string

from pandas.io.json import json_normalize
from pyspark.sql.functions import explode, flatten
import json

from pyspark.sql.functions import explode, flatten

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


##Mount G Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##Load data to Spark Dataframe

In [ ]:
path = '/content/drive/My Drive/Data/Data spliting/mergedReviewsAndOfferings in parts/jqSam.json'
# path = '/content/drive/My Drive/merged.json'
df_data = spark.read.json(path)

##Rearrange Columns

In [ ]:
df_data = df_data.select("id","hotel_id","hotel_name","overall_rating", "text", "title", "author_id")

##Sort Columns

In [ ]:
df_data = df_data.orderBy('id', ascending=True)

In [ ]:
# df_data = df_data.orderBy('text', ascending=False, inline = True)

In [ ]:
df_data.show(4)

+------+--------+--------------------+--------------+--------------------+--------------------+--------------------+
|    id|hotel_id|          hotel_name|overall_rating|                text|               title|           author_id|
+------+--------+--------------------+--------------+--------------------+--------------------+--------------------+
|683156|  220216|Beacon Hill Hotel...|             5|The rooms here ar...|“great hotel, gre...|C0FE2026E16771D2C...|
|692745|  113317|Casablanca Hotel ...|             5|I travelled from ...|“absolutely delig...|8C17DDA718DC7815A...|
|692780|  100540|      MarQueen Hotel|             2|Sorry this is a b...|“Marqueen, overra...|75EACF34E876B7366...|
|692844|   80133|Handlery Hotel Sa...|             1|Rooms have been r...|“a motel ' dreami...|35FBC6EA978B4AE51...|
+------+--------+--------------------+--------------+--------------------+--------------------+--------------------+
only showing top 4 rows



#Tokenizing

In [ ]:
def word_TokenizeFunct(x):
    splitted = [word for line in x for word in line.split()]
    return splitted

def removeStopWordsFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence

def removePunctuationsFunct(x):
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x]
    filtered_space = [s for s in filtered if s] #remove empty space
    return filtered

def lemmatizationFunct(x):
    nltk.download('wordnet')
    lemmatizer = WordNetLemmatizer()
    finalLem = [lemmatizer.lemmatize(s) for s in x]
    return finalLem

# def removeNonEnglishWordsFunct(arr):
#     words = set(nltk.corpus.words.words())
#     filteredSentence = ""
#     for token in arr:
#         print("now:"+ token)
#         res = ''
#         res = " ".join(w for w in nltk.wordpunct_tokenize(token) \
#                               if w.lower() in words or not w.isalpha())
#         if res != '':
#           filteredSentence = filteredSentence + res + " "
#     return filteredSentence.strip()

def tokenizeColumn(text):
    textProcessing = text.lower()
    # textProcessing = removeNonEnglishWordsFunct(textProcessing)
    textProcessing = nltk.sent_tokenize(textProcessing)
    textProcessing = [word for line in textProcessing for word in line.split()]
    textProcessing = removeStopWordsFunct(textProcessing)
    textProcessing = removePunctuationsFunct(textProcessing)
    textProcessing = lemmatizationFunct(textProcessing)
    textProcessing = ["'" + item + "'" for item in textProcessing]
    return textProcessing #[text] text array


udf_tokenize_column = udf(tokenizeColumn, ArrayType(StringType()))

In [ ]:
df_data = df_data.withColumn('tokenized', udf_tokenize_column(df_data.text))

In [ ]:
df_data.select('tokenized').take(1)

[Row(tokenized=["'room'", "'palatial'", "'theyre'", "'comfortable'", "'flatscreen'", "'tv'", "'nice'", "'touch'", "'also'", "'hotel'", "'restaurant'", "'fabulous'", "'breakfast'", "'terrific'", "'cant'", "'beat'", "'charles'", "'street'"])]

#Thesaurus

In [ ]:
thesaurus={
           "service":["service","waiter","staff","server"],
           "room":["bed","room","bathroom"],
           "shopping":["mall","shopping","store","market"],
           "cleanliness":["dirty","grubby","clean","neat"],
           "food":["eat","dishes","dinner","food","breakfast","delicious","meal","restaurant","lunch"],
           "value":["price","cheap","worth","money","expensive","pay"],
           "transportation":["subway","stop","transportation","bus"],
           "family":["mother","friend","father","daughter","husband","child","son","kid","wife"],
           "friends":["mother","friend","father","daughter","husband","child","son","kid","wife"],
           "fitness":["pool","fitness","gym","swimming","spa"],
           "environment":["modern","environment","comfortable","classical"]
           }

thesaurus_keys = list(thesaurus.keys())

print(thesaurus_keys)

['service', 'room', 'shopping', 'cleanliness', 'food', 'value', 'transportation', 'family', 'friends', 'fitness', 'environment']


In [ ]:
def makeThesaurus(tokensArray):
  thesaurus_arr = []
  for token in tokensArray: #token = staff
    for thes_key in thesaurus_keys: #key = service
      keyItemsArr = thesaurus[thes_key] #thesaurus[key] # keyItemsArr = ["service","waiter","staff","server"]
      if token in  str(keyItemsArr):
        thesaurus_arr.append(thes_key)
  thesaurus_set=list(set(thesaurus_arr))
  return thesaurus_set #[text] text array


udf_thesaurus_column = udf(makeThesaurus, ArrayType(StringType()))

In [ ]:
df_data = df_data.withColumn('thesaurus', udf_thesaurus_column(df_data.tokenized))

In [ ]:
df_test = df_data

In [ ]:
#filtering empty arrrays from thesaurus column
from pyspark.sql import functions as F

df_data =  df_test.filter(F.size('thesaurus') > 0)

In [ ]:
# df_data.select('id', 'text').show(10, False)
df_data.select('id', 'tokenized').show(10, False)
df_data.select('id', 'thesaurus').show(10, False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Similatiry Computation


##Jaccard

In [ ]:
# active_user_thesaurus=['room','family','shopping','environment']

def Jaccard_Similarity(previous_user_thesaurus):
  # for row in thesaurusArray:
  active = set(['cleanliness','environment','food','shopping','fitness'])
  prev_usr_set = set(previous_user_thesaurus)
  ###len(s1.intersection(s2)) / len(s1.union(s2))
  x= len(active.intersection(prev_usr_set)) / len(active.union(prev_usr_set))
  return round(x, 2)

udf_Jaccard_Similarity_column = udf(Jaccard_Similarity, returnType= FloatType())

In [ ]:
df_data = df_data.withColumn('jaccard_sim',udf_Jaccard_Similarity_column(df_data.thesaurus))

In [ ]:
# df_data.show(3)
df_data.printSchema()

root
 |-- id: long (nullable = true)
 |-- hotel_id: long (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- overall_rating: long (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- tokenized: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- thesaurus: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- jaccard_sim: float (nullable = true)



In [ ]:
# df_small = df_data.select('id','hotel_name','jaccard_sim').orderBy('jaccard_sim', ascending=False)
df_small = df_data.select('id','hotel_name','overall_rating','thesaurus','jaccard_sim','author_id')


In [ ]:
df_small.show()

+------+--------------------+--------------+--------------------+-----------+--------------------+
|    id|          hotel_name|overall_rating|           thesaurus|jaccard_sim|           author_id|
+------+--------------------+--------------+--------------------+-----------+--------------------+
|683156|Beacon Hill Hotel...|             5|[food, room, envi...|       0.33|C0FE2026E16771D2C...|
|692745|Casablanca Hotel ...|             5|    [room, shopping]|       0.17|8C17DDA718DC7815A...|
|692780|      MarQueen Hotel|             2|[family, room, va...|       0.18|75EACF34E876B7366...|
|692844|Handlery Hotel Sa...|             1|[family, room, cl...|       0.33|35FBC6EA978B4AE51...|
|692847|The Westin Los An...|             5|[room, cleanlines...|       0.38|5BE26753AC8336073...|
|692892|          Hotel Diva|             4| [room, cleanliness]|       0.17|E01681A50DC19FFA1...|
|692980|Hampton Inn Houst...|             4|[cleanliness, foo...|        0.5|38078EB1B07129384...|
|693626|Ri

##Simpson

In [ ]:
def simpson_similarity(previous_user_thesaurus):
    activeuser = set(['cleanliness','environment','food','shopping','fitness'])
    prev_usr_set = set(previous_user_thesaurus)
    n= min(len(prev_usr_set),len(activeuser))
    print(n)
    a= len((prev_usr_set).intersection(activeuser))
    print(a)
    if n !=0:
      b = a/n
    else:
      b=0.0

    return round(b,2)


# prev_usr_set = set(['value', 'fitness', 'room', 'service','environment'])
# simpson_similarity(prev_usr_set)

udf_Simpson_Similarity_column = udf(simpson_similarity, returnType= FloatType())

In [ ]:
df_simp=df_data
df_simp = df_simp.withColumn('simpson_sim',udf_Simpson_Similarity_column(df_simp.thesaurus))

In [ ]:
df_simp.printSchema()

root
 |-- id: long (nullable = true)
 |-- hotel_id: long (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- overall_rating: long (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- tokenized: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- thesaurus: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- jaccard_sim: float (nullable = true)
 |-- simpson_sim: float (nullable = true)



In [ ]:
df_simp.show()

+------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+
|    id|hotel_id|          hotel_name|overall_rating|                text|               title|           author_id|           tokenized|           thesaurus|jaccard_sim|simpson_sim|
+------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+
|683156|  220216|Beacon Hill Hotel...|             5|The rooms here ar...|“great hotel, gre...|C0FE2026E16771D2C...|['room', 'palatia...|[food, room, envi...|       0.33|       0.67|
|692745|  113317|Casablanca Hotel ...|             5|I travelled from ...|“absolutely delig...|8C17DDA718DC7815A...|['travelled', 'sp...|    [room, shopping]|       0.17|        0.5|
|692780|  100540|      MarQueen Hotel|             2|Sorry this is a b...|“Marqueen, 

##Cosine

In [ ]:
df_concat=df_simp.select('author_id','hotel_name','overall_rating','thesaurus','jaccard_sim')
df_concat.show()

+--------------------+--------------------+--------------+--------------------+-----------+
|           author_id|          hotel_name|overall_rating|           thesaurus|jaccard_sim|
+--------------------+--------------------+--------------+--------------------+-----------+
|C0FE2026E16771D2C...|Beacon Hill Hotel...|             5|[food, room, envi...|       0.33|
|8C17DDA718DC7815A...|Casablanca Hotel ...|             5|    [room, shopping]|       0.17|
|75EACF34E876B7366...|      MarQueen Hotel|             2|[family, room, va...|       0.18|
|35FBC6EA978B4AE51...|Handlery Hotel Sa...|             1|[family, room, cl...|       0.33|
|5BE26753AC8336073...|The Westin Los An...|             5|[room, cleanlines...|       0.38|
|E01681A50DC19FFA1...|          Hotel Diva|             4| [room, cleanliness]|       0.17|
|38078EB1B07129384...|Hampton Inn Houst...|             4|[cleanliness, foo...|        0.5|
|8916B7F8AA251A7BD...|Ritz-Carlton Batt...|             5|[room, value, env...| 

In [ ]:
df_grouped=df_concat.groupby("author_id").count()
df_grouped.show()

+--------------------+-----+
|           author_id|count|
+--------------------+-----+
|C91CAB1743DBF417B...|    1|
|A1148775F67E1100C...|    1|
|051EEA2910C52D43E...|    4|
|10439AB61C9F45FE8...|    1|
|6C627DE9C83E5698D...|    2|
|DF720B78F6BAA12BF...|    1|
|9D5201F660844CEBB...|    3|
|D1785EFDF52D479DB...|    1|
|E518ED8D9EA816739...|    1|
|3FAB4597BA6BE0627...|    1|
|552CAB2BC55E6998B...|    1|
|169314F596C4637E3...|    2|
|E485E3B4D0B7DBDEA...|    1|
|DBFD921B672524124...|    1|
|1D724D8AF14861402...|    1|
|BF452B9F8734327BE...|    1|
|6BC207B670737B163...|    1|
|3BFEC984DBB46FF1D...|    1|
|10DC208BF71A051AC...|    1|
|8B2144C81DE14364F...|    2|
+--------------------+-----+
only showing top 20 rows



In [ ]:
from pyspark.sql import functions as F
df_grouped = df_concat.groupby('author_id').agg(F.collect_list(df_concat["thesaurus"]).alias('all_thesaurus'))
df_grouped.show()

+--------------------+--------------------+
|           author_id|       all_thesaurus|
+--------------------+--------------------+
|C91CAB1743DBF417B...|[[room, value, fi...|
|A1148775F67E1100C...|[[room, cleanline...|
|051EEA2910C52D43E...|[[family, room, v...|
|10439AB61C9F45FE8...|[[room, value, sh...|
|6C627DE9C83E5698D...|[[room], [room, e...|
|DF720B78F6BAA12BF...|[[service, room, ...|
|9D5201F660844CEBB...|[[room, value, sh...|
|D1785EFDF52D479DB...|[[room, value, fi...|
|E518ED8D9EA816739...|      [[food, room]]|
|3FAB4597BA6BE0627...|      [[food, room]]|
|552CAB2BC55E6998B...|[[family, room, v...|
|169314F596C4637E3...|[[food, room, cle...|
|E485E3B4D0B7DBDEA...|[[room, environme...|
|DBFD921B672524124...|[[service, cleanl...|
|1D724D8AF14861402...|[[family, room, c...|
|BF452B9F8734327BE...|[[food, room, env...|
|6BC207B670737B163...|[[room, value, en...|
|3BFEC984DBB46FF1D...|[[family, room, f...|
|10DC208BF71A051AC...|[[family, room, c...|
|8B2144C81DE14364F...|[[room, va

In [ ]:
from pyspark.sql.functions import col
testDF4 = df_grouped.select(col("author_id").alias("author_id2"), col("all_thesaurus").alias("all_reviews"))
testDF4.show()

+--------------------+--------------------+
|          author_id2|         all_reviews|
+--------------------+--------------------+
|C91CAB1743DBF417B...|[[room, value, fi...|
|A1148775F67E1100C...|[[room, cleanline...|
|051EEA2910C52D43E...|[[family, room, v...|
|10439AB61C9F45FE8...|[[room, value, sh...|
|6C627DE9C83E5698D...|[[room], [room, e...|
|DF720B78F6BAA12BF...|[[service, room, ...|
|9D5201F660844CEBB...|[[room, value, sh...|
|D1785EFDF52D479DB...|[[room, value, fi...|
|E518ED8D9EA816739...|      [[food, room]]|
|3FAB4597BA6BE0627...|      [[food, room]]|
|552CAB2BC55E6998B...|[[family, room, v...|
|169314F596C4637E3...|[[food, room, cle...|
|E485E3B4D0B7DBDEA...|[[room, environme...|
|DBFD921B672524124...|[[service, cleanl...|
|1D724D8AF14861402...|[[family, room, c...|
|BF452B9F8734327BE...|[[food, room, env...|
|6BC207B670737B163...|[[room, value, en...|
|3BFEC984DBB46FF1D...|[[family, room, f...|
|10DC208BF71A051AC...|[[family, room, c...|
|8B2144C81DE14364F...|[[room, va

###PPK weight vectors

In [ ]:
def weight_vectors(thesaurus_list):
  import math
  keyword_list=['service','room','shopping','cleanliness','food','value','transportation','family','fitness','environment']
  count_list=[]
  # a = np.zeros(len(keyword_list))

  number_of_reviews=len(thesaurus_list)
  print(number_of_reviews)

  total_keywords_of_user=0
  for listElem in thesaurus_list:
    total_keywords_of_user += len(listElem)

  print(total_keywords_of_user)

  keyword_count=0;

  for key in keyword_list:
    for items in thesaurus_list:
      for item in items:
       if item == key:
        keyword_count+=1
    count_list.append(keyword_count)
    keyword_count=0

  tf_list=[]
  idf_list=[]
  tf_idf_list=[]


  for i in range(len(count_list)):
    a=count_list[i]
    if total_keywords_of_user != 0:
      tf=float(a/total_keywords_of_user)
      tf=round(tf,3)
      tf_list.append(tf)
    else:
      tf_list.append(0.0)



  print("Count list")
  print(count_list)
  print("TF list")
  print(tf_list)
  print("IDF list")
  print(idf_list)
  print("TF-IDF list")
  print(tf_idf_list)

  return tf_list



# thes=[['environment', 'food', 'shopping', 'room'], ['food', 'room', 'friends', 'shopping', 'family', 'value', 'service', 'fitness'], ['cleanliness', 'food', 'room', 'service'], ['food', 'room', 'service'], ['food', 'room', 'cleanliness', 'friends', 'value', 'family', 'transportation', 'shopping', 'service']]
# weight_vectors(thes)

udf_TF_column = udf(weight_vectors, returnType= ArrayType(FloatType()))


In [ ]:
testDF4 = testDF4.withColumn('weight_of_previous_user',udf_TF_column(testDF4.all_reviews))
testDF4.show()

+--------------------+--------------------+-----------------------+
|          author_id2|         all_reviews|weight_of_previous_user|
+--------------------+--------------------+-----------------------+
|C91CAB1743DBF417B...|[[room, value, fi...|   [0.2, 0.2, 0.0, 0...|
|A1148775F67E1100C...|[[room, cleanline...|   [0.125, 0.125, 0....|
|051EEA2910C52D43E...|[[family, room, v...|   [0.167, 0.167, 0....|
|10439AB61C9F45FE8...|[[room, value, sh...|   [0.0, 0.333, 0.33...|
|6C627DE9C83E5698D...|[[room], [room, e...|   [0.0, 0.667, 0.0,...|
|DF720B78F6BAA12BF...|[[service, room, ...|   [0.333, 0.333, 0....|
|9D5201F660844CEBB...|[[room, value, sh...|   [0.214, 0.214, 0....|
|D1785EFDF52D479DB...|[[room, value, fi...|   [0.2, 0.2, 0.0, 0...|
|E518ED8D9EA816739...|      [[food, room]]|   [0.0, 0.5, 0.0, 0...|
|3FAB4597BA6BE0627...|      [[food, room]]|   [0.0, 0.5, 0.0, 0...|
|552CAB2BC55E6998B...|[[family, room, v...|   [0.2, 0.2, 0.0, 0...|
|169314F596C4637E3...|[[food, room, cle...|   [0

###APK weight vectors

In [ ]:
def CosineSimTF(prev_user_weight):

  keyword_list=['service','room','shopping','cleanliness','food','value','transportation','family','fitness','environment']
  apk = "cleanliness:3,environment:4,food:5,shopping:2,fitness:5"

  words=apk.split(",")

  # print(words)

  apkMatrix = [[0 for i in range(len(words))] for j in range(len(words))]
  # print(apkMatrix)

  apkMatrixColumnSum=[0]*(len(words))
  # print(apkMatrixColumnSum)

  num=words[0].split(":")
  # print(num[1])


  for j in range(len(words)):
    splitString=words[j].split(":")
    valuej=int(splitString[1])
    for i in range(len(words)):
      splits=words[i].split(":")
      valuei=int(splits[1])
      if valuej==valuei:
        apkMatrix[i][j]=1
      elif valuei>valuej:
        temp=valuej/valuei
        apkMatrix[i][j]=1/temp
      elif valuej>valuei:
        apkMatrix[i][j]=valuei/valuej

  # print(apkMatrix)

  for j in range(len(words)):
    for i in range(len(words)):
      apkMatrixColumnSum[j]+=apkMatrix[i][j]


  for k in range(len(words)):
    # print(apkMatrixColumnSum[k])
    for l in range(len(words)):
      apkMatrix[l][k]=apkMatrix[l][k]/float(apkMatrixColumnSum[k])

  # print(apkMatrix)

  weight_array=[0]*(len(words))

  for j in range(len(words)):
    for i in range(len(words)):
      weight_array[j]+=apkMatrix[j][i]
    weight_array[j]=weight_array[j]/(len(words))
    weight_array[j]=round(weight_array[j],3)

# print(apkMatrixColumnSum)
  # print(weight_array)

  apk_vector=[0.0]*10
  # print(apk_vector)
  n=len(words)

  for j in range(len(keyword_list)):
    for i in range(n):
      x=words[i].split(":")
      keyword=x[0]
    # print(keyword_list[j])
      if keyword_list[j] == keyword:
        val=weight_array[i]
        apk_vector[j]=val

  print(apk_vector)
  import numpy as np

  b=np.array(prev_user_weight)
  a=np.array(apk_vector)

  x = np.dot(a, b)
  x = round(x, 2)
  print(x)
  typX = type(x).__name__
  xStr = str(x)
  y= np.sqrt(np.dot(a, a) * np.dot(b, b))
  y = round(y, 2)
  print(y)
  typY = type(y).__name__
  yStr = str(y)

  z = x/y
  z = round(z, 2)
  z = np.float32(z)
  zStr = str(z)
  zFloat = float(zStr)

  return zFloat




# weight=[0.143, 0.286, 0.0, 0.143, 0.286, 0.0, 0.0, 0.0, 0.0, 0.143]
# CosineSimTF(weight)

udf_Cosine_Sim_column = udf(CosineSimTF, returnType= FloatType())


In [ ]:
testDF4 = testDF4.withColumn('Cosine_Sim',udf_Cosine_Sim_column(testDF4.weight_of_previous_user))
testDF4.show()

+--------------------+--------------------+-----------------------+----------+
|          author_id2|         all_reviews|weight_of_previous_user|Cosine_Sim|
+--------------------+--------------------+-----------------------+----------+
|C91CAB1743DBF417B...|[[room, value, fi...|   [0.2, 0.2, 0.0, 0...|      0.52|
|A1148775F67E1100C...|[[room, cleanline...|   [0.125, 0.125, 0....|      0.59|
|051EEA2910C52D43E...|[[family, room, v...|   [0.167, 0.167, 0....|       0.5|
|10439AB61C9F45FE8...|[[room, value, sh...|   [0.0, 0.333, 0.33...|      0.11|
|6C627DE9C83E5698D...|[[room], [room, e...|   [0.0, 0.667, 0.0,...|       0.2|
|DF720B78F6BAA12BF...|[[service, room, ...|   [0.333, 0.333, 0....|      0.11|
|9D5201F660844CEBB...|[[room, value, sh...|   [0.214, 0.214, 0....|      0.33|
|D1785EFDF52D479DB...|[[room, value, fi...|   [0.2, 0.2, 0.0, 0...|      0.52|
|E518ED8D9EA816739...|      [[food, room]]|   [0.0, 0.5, 0.0, 0...|      0.39|
|3FAB4597BA6BE0627...|      [[food, room]]|   [0.0, 

In [ ]:
testDF4.printSchema()

root
 |-- author_id2: string (nullable = true)
 |-- all_reviews: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- weight_of_previous_user: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Cosine_Sim: float (nullable = true)



In [ ]:
from pyspark.sql.functions import explode
testDF4.select(testDF4.author_id2,testDF4.weight_of_previous_user,testDF4.Cosine_Sim,explode(testDF4.all_reviews))
testDF4.show()

+--------------------+--------------------+-----------------------+----------+
|          author_id2|         all_reviews|weight_of_previous_user|Cosine_Sim|
+--------------------+--------------------+-----------------------+----------+
|C91CAB1743DBF417B...|[[room, value, fi...|   [0.2, 0.2, 0.0, 0...|      0.52|
|A1148775F67E1100C...|[[room, cleanline...|   [0.125, 0.125, 0....|      0.59|
|051EEA2910C52D43E...|[[family, room, v...|   [0.167, 0.167, 0....|       0.5|
|10439AB61C9F45FE8...|[[room, value, sh...|   [0.0, 0.333, 0.33...|      0.11|
|6C627DE9C83E5698D...|[[room], [room, e...|   [0.0, 0.667, 0.0,...|       0.2|
|DF720B78F6BAA12BF...|[[service, room, ...|   [0.333, 0.333, 0....|      0.11|
|9D5201F660844CEBB...|[[room, value, sh...|   [0.214, 0.214, 0....|      0.33|
|D1785EFDF52D479DB...|[[room, value, fi...|   [0.2, 0.2, 0.0, 0...|      0.52|
|E518ED8D9EA816739...|      [[food, room]]|   [0.0, 0.5, 0.0, 0...|      0.39|
|3FAB4597BA6BE0627...|      [[food, room]]|   [0.0, 

In [ ]:
df_cosine = df_simp.join(testDF4, df_simp.author_id == testDF4.author_id2)
df_cosine.show()

+------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+-----------------------+----------+
|    id|hotel_id|          hotel_name|overall_rating|                text|               title|           author_id|           tokenized|           thesaurus|jaccard_sim|simpson_sim|          author_id2|         all_reviews|weight_of_previous_user|Cosine_Sim|
+------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+-----------------------+----------+
|683156|  220216|Beacon Hill Hotel...|             5|The rooms here ar...|“great hotel, gre...|C0FE2026E16771D2C...|['room', 'palatia...|[food, room, envi...|       0.33|       0.67|C0FE2026E16771D2C...|[[food, room, env

In [ ]:
print(df_cosine.count())

8618


#Calculating Personalized Rating

##Filter Threshold

In [ ]:
df_filtered_sim = df_cosine.filter(col("Cosine_Sim") > 0.5)
# df_filtered_sim = df_filter.filter((col("jaccard_sim") > 0.7) & (col("simpson_sim")>0.7) & (col("cosine_sim")>0.7))
df_filtered_sim.show()

+------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+-----------------------+----------+
|    id|hotel_id|          hotel_name|overall_rating|                text|               title|           author_id|           tokenized|           thesaurus|jaccard_sim|simpson_sim|          author_id2|         all_reviews|weight_of_previous_user|Cosine_Sim|
+------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+-----------------------+----------+
|683156|  220216|Beacon Hill Hotel...|             5|The rooms here ar...|“great hotel, gre...|C0FE2026E16771D2C...|['room', 'palatia...|[food, room, envi...|       0.33|       0.67|C0FE2026E16771D2C...|[[food, room, env

In [ ]:
print(df_filtered_sim.count())

1660


In [ ]:
from pyspark.sql import functions as F
agg_df = df_filtered_sim.groupBy('author_id').agg(F.collect_list('overall_rating').alias('collected_ratings'))
agg_df.show()
# agg_df = agg_df.withColumn('new', fun('collected_ratings'))

+--------------------+-----------------+
|           author_id|collected_ratings|
+--------------------+-----------------+
|C91CAB1743DBF417B...|              [4]|
|A1148775F67E1100C...|              [5]|
|D1785EFDF52D479DB...|              [5]|
|E485E3B4D0B7DBDEA...|              [3]|
|BF452B9F8734327BE...|              [4]|
|38F0C4D0D9B429BD3...|              [5]|
|50321281A9B2D1A59...|              [2]|
|28043347C8B3104E4...|              [5]|
|765B70BCC14881041...|              [4]|
|A8C4C119EAFB17523...|              [3]|
|5390D37CF79707CBD...|              [3]|
|1E38B217001359D58...|              [4]|
|A28D82457755229BC...|           [5, 4]|
|DADBD3DD437872175...|              [3]|
|F164A015A7AFC8430...|              [4]|
|8658A3FFC7E6223F5...|              [4]|
|B1C95B4EBC33F9F69...|           [4, 4]|
|024147F852BC55FF0...|              [4]|
|FECF9197299DEC7D3...|              [5]|
|3AB336BFE841EF2CD...|              [5]|
+--------------------+-----------------+
only showing top

In [ ]:
def mean_rating(rating_list):
  number_of_ratings=0
  for listElem in rating_list:
    number_of_ratings+=1
  print(number_of_ratings)
  sum=0

  for i in range(len(rating_list)):
    sum+=rating_list[i]
  print(sum)

  mean=sum/number_of_ratings

  return mean

# list= [5, 4]
# mean_rating(list)

udf_mean_ratings_column = udf(mean_rating, returnType= FloatType())


In [ ]:
agg_df = agg_df.withColumn('MeanRatings',udf_mean_ratings_column(agg_df.collected_ratings))

In [ ]:
agg_df.show()

+--------------------+-----------------+-----------+
|           author_id|collected_ratings|MeanRatings|
+--------------------+-----------------+-----------+
|C91CAB1743DBF417B...|              [4]|        4.0|
|A1148775F67E1100C...|              [5]|        5.0|
|D1785EFDF52D479DB...|              [5]|        5.0|
|E485E3B4D0B7DBDEA...|              [3]|        3.0|
|BF452B9F8734327BE...|              [4]|        4.0|
|38F0C4D0D9B429BD3...|              [5]|        5.0|
|50321281A9B2D1A59...|              [2]|        2.0|
|28043347C8B3104E4...|              [5]|        5.0|
|765B70BCC14881041...|              [4]|        4.0|
|A8C4C119EAFB17523...|              [3]|        3.0|
|5390D37CF79707CBD...|              [3]|        3.0|
|1E38B217001359D58...|              [4]|        4.0|
|A28D82457755229BC...|           [5, 4]|        4.5|
|DADBD3DD437872175...|              [3]|        3.0|
|F164A015A7AFC8430...|              [4]|        4.0|
|8658A3FFC7E6223F5...|              [4]|      

In [ ]:
agg_df.printSchema()
from pyspark.sql.functions import explode
df_explode = agg_df.select(agg_df.author_id,agg_df.MeanRatings,explode(agg_df.collected_ratings))
df_explode.show()

root
 |-- author_id: string (nullable = true)
 |-- collected_ratings: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- MeanRatings: float (nullable = true)

+--------------------+-----------+---+
|           author_id|MeanRatings|col|
+--------------------+-----------+---+
|C91CAB1743DBF417B...|        4.0|  4|
|A1148775F67E1100C...|        5.0|  5|
|D1785EFDF52D479DB...|        5.0|  5|
|E485E3B4D0B7DBDEA...|        3.0|  3|
|BF452B9F8734327BE...|        4.0|  4|
|38F0C4D0D9B429BD3...|        5.0|  5|
|50321281A9B2D1A59...|        2.0|  2|
|28043347C8B3104E4...|        5.0|  5|
|765B70BCC14881041...|        4.0|  4|
|A8C4C119EAFB17523...|        3.0|  3|
|5390D37CF79707CBD...|        3.0|  3|
|1E38B217001359D58...|        4.0|  4|
|A28D82457755229BC...|        4.5|  5|
|A28D82457755229BC...|        4.5|  4|
|DADBD3DD437872175...|        3.0|  3|
|F164A015A7AFC8430...|        4.0|  4|
|8658A3FFC7E6223F5...|        4.0|  4|
|B1C95B4EBC33F9F69...|        4.0|  4

In [ ]:
joinedDF = df_filtered_sim.join(agg_df, df_filtered_sim.author_id == agg_df.author_id)

In [ ]:
joinedDF.show()

+-------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+-----------------------+----------+--------------------+-----------------+-----------+
|     id|hotel_id|          hotel_name|overall_rating|                text|               title|           author_id|           tokenized|           thesaurus|jaccard_sim|simpson_sim|          author_id2|         all_reviews|weight_of_previous_user|Cosine_Sim|           author_id|collected_ratings|MeanRatings|
+-------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+-----------------------+----------+--------------------+-----------------+-----------+
|2268680|  115644|Chancellor Hotel ...|             5|We stayed 

##Summation of overall rating

In [ ]:
import pyspark.sql.functions as F

sum_of_overall_rating= joinedDF.agg(F.sum("overall_rating")).collect()[0][0]
print(sum_of_overall_rating)

6625


##Number of filtered rows

In [ ]:
sum=joinedDF.count()
print(sum)

1660


##Calculating Average Ratings

In [ ]:
average_ratings=sum_of_overall_rating// sum
print(average_ratings)

3


##Summation of Similarity

In [ ]:
Summation_of_Sim_Jaccard= joinedDF.agg(F.sum("jaccard_sim")).collect()[0][0]
print(Summation_of_Sim_Jaccard)

596.6400035619736


In [ ]:
Summation_of_Sim_Simpson= joinedDF.agg(F.sum("simpson_sim")).collect()[0][0]
print(Summation_of_Sim_Simpson)

1011.6600214391947


In [ ]:
Summation_of_Sim_Cosine= joinedDF.agg(F.sum("Cosine_Sim")).collect()[0][0]
print(Summation_of_Sim_Cosine)

993.7499883770943


##Calculating normalizing factor(k)

In [ ]:
# normalizing_factor= 1/Summation_Of_Sim
# print(normalizing_factor)

# normalizing_factor_simp= 1/Summation_Of_Sim_Simpson
# print(normalizing_factor_simp)

# normalizing_factor_cosine= 1/Summation_Of_Sim_Cosine
# print(normalizing_factor_cosine)

##Applying PR formula

In [ ]:
# JACCARD
def personalized_rating1(rating,mean_ratings):
  x=average_ratings+((Summation_of_Sim_Jaccard*(rating-mean_ratings))/Summation_of_Sim_Jaccard)
  return round(x, 3)
udf_PR_column = udf(personalized_rating1, returnType= FloatType())

# SIMPSON
def personalized_rating2(rating,mean_ratings):
  x=average_ratings+((Summation_of_Sim_Simpson*(rating-mean_ratings))/Summation_of_Sim_Simpson)
  return round(x, 3)

udf_PR2_column = udf(personalized_rating2, returnType= FloatType())

# //COSINE
def personalized_rating3(rating,mean_ratings):
  x=average_ratings+((Summation_of_Sim_Cosine*(rating-mean_ratings))/Summation_of_Sim_Cosine)
  return round(x, 3)

udf_PR3_column = udf(personalized_rating3, returnType= FloatType())


In [ ]:
joinedDF = joinedDF.withColumn('PR_Jaccard',udf_PR_column(joinedDF.overall_rating,joinedDF.MeanRatings))
joinedDF = joinedDF.withColumn('PR_Simpson',udf_PR2_column(joinedDF.overall_rating,joinedDF.MeanRatings))
joinedDF = joinedDF.withColumn('PR_Cosine',udf_PR3_column(joinedDF.overall_rating,joinedDF.MeanRatings))


In [ ]:
joinedDF.orderBy('jaccard_sim', ascending=False).select("hotel_name","overall_rating","thesaurus","jaccard_sim","Cosine_Sim","simpson_sim").show(truncate=False)

+------------------------------------------------+--------------+------------------------------------------------------------------+-----------+----------+-----------+
|hotel_name                                      |overall_rating|thesaurus                                                         |jaccard_sim|Cosine_Sim|simpson_sim|
+------------------------------------------------+--------------+------------------------------------------------------------------+-----------+----------+-----------+
|Pointe Hilton Tapatio Cliffs Resort             |4             |[fitness, food, environment, cleanliness]                         |0.8        |0.96      |1.0        |
|Country Inn & Suites By Carlson, San Diego North|3             |[room, cleanliness, environment, fitness, shopping, service, food]|0.71       |0.78      |1.0        |
|Econo Lodge Austin                              |4             |[room, cleanliness, value, environment, fitness, shopping, food]  |0.71       |0.76      |1.0  

In [ ]:
joinedDF.orderBy('Cosine_Sim', ascending=False).select("hotel_name","overall_rating","thesaurus","jaccard_sim","Cosine_Sim","simpson_sim").show(truncate=False)

+---------------------------------------------+--------------+--------------------------------------------------------+-----------+----------+-----------+
|hotel_name                                   |overall_rating|thesaurus                                               |jaccard_sim|Cosine_Sim|simpson_sim|
+---------------------------------------------+--------------+--------------------------------------------------------+-----------+----------+-----------+
|Pointe Hilton Tapatio Cliffs Resort          |4             |[fitness, food, environment, cleanliness]               |0.8        |0.96      |1.0        |
|Embassy Suites Hotel Denver - Aurora         |4             |[room, cleanliness, environment, fitness, food]         |0.67       |0.86      |0.8        |
|BEST WESTERN PLUS Blue Sea Lodge             |5             |[room, cleanliness, environment, fitness, food]         |0.67       |0.86      |0.8        |
|Econo Lodge                                  |2             |[room, c

In [ ]:
joinedDF.orderBy('simpson_sim', ascending=False).select("hotel_name","overall_rating","thesaurus","jaccard_sim","Cosine_Sim","simpson_sim").show(truncate=False)

+---------------------------------------------+--------------+----------------------------------------------------------------------------------------------------------+-----------+----------+-----------+
|hotel_name                                   |overall_rating|thesaurus                                                                                                 |jaccard_sim|Cosine_Sim|simpson_sim|
+---------------------------------------------+--------------+----------------------------------------------------------------------------------------------------------+-----------+----------+-----------+
|Crowne Plaza Jacksonville Riverfront Hotel   |5             |[fitness, food]                                                                                           |0.4        |0.79      |1.0        |
|Affinia Manhattan                            |5             |[food, cleanliness]                                                                                       |0.4        

#Accuracy Calculation

In [ ]:
joinedDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- hotel_id: long (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- overall_rating: long (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- tokenized: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- thesaurus: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- jaccard_sim: float (nullable = true)
 |-- simpson_sim: float (nullable = true)
 |-- author_id2: string (nullable = true)
 |-- all_reviews: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- weight_of_previous_user: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Cosine_Sim: float (nullable = true)
 |-- author_id: string (nullable = true)
 |-- collected_ratings: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |--

In [ ]:
# df_accuracy=df_final
joinedDF= joinedDF.withColumn('Jaccard[qi-pi]',(joinedDF['overall_rating'] - joinedDF['PR_Jaccard']))
joinedDF = joinedDF.withColumn('Simpson[qi-pi]', ( joinedDF['overall_rating'] - joinedDF['PR_Simpson'] ))
joinedDF = joinedDF.withColumn('Cosine[qi-pi]', ( joinedDF['overall_rating'] - joinedDF['PR_Cosine'] ))

In [ ]:
joinedDF.orderBy('PR_Cosine',ascending=False).show()

+-------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+-----------------------+----------+--------------------+--------------------+-----------+----------+----------+---------+--------------+--------------+-------------+
|     id|hotel_id|          hotel_name|overall_rating|                text|               title|           author_id|           tokenized|           thesaurus|jaccard_sim|simpson_sim|          author_id2|         all_reviews|weight_of_previous_user|Cosine_Sim|           author_id|   collected_ratings|MeanRatings|PR_Jaccard|PR_Simpson|PR_Cosine|Jaccard[qi-pi]|Simpson[qi-pi]|Cosine[qi-pi]|
+-------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+--------------------+--------------

In [ ]:
result_acc1= joinedDF.agg(F.sum("Jaccard[qi-pi]")).collect()[0][0]
print(result_acc1)
result_acc2= joinedDF.agg(F.sum("Simpson[qi-pi]")).collect()[0][0]
print(result_acc2)
result_acc3= joinedDF.agg(F.sum("Cosine[qi-pi]")).collect()[0][0]
print(result_acc3)

1645.0020005106926
1645.0020005106926
1645.0020005106926


In [ ]:
MAE_Jaccard= result_acc1/sum
print(MAE_Jaccard)
MAE_Simp=result_acc2/sum
print(MAE_Simp)
MAE_Cosine=result_acc3/sum
print(MAE_Cosine)

0.99096506054861
0.99096506054861
0.99096506054861


In [ ]:
x= sum_of_overall_rating/sum
NMAE_Jaccard=MAE_Jaccard/x
print(NMAE_Jaccard)
NMAE_Simp=MAE_Simp/x
print(NMAE_Simp)
NMAE_Cosine=MAE_Cosine/x
print(NMAE_Cosine)

0.24830218875633098
0.24830218875633098
0.24830218875633098
